In [1]:
pip install opencv-contrib-python


     ---------------------------------------- 45.5/45.5 MB 2.1 MB/s eta 0:00:00


In [2]:
import cv2
print(dir(cv2.face))

['BIF', 'BIF_create', 'BasicFaceRecognizer', 'EigenFaceRecognizer', 'EigenFaceRecognizer_create', 'FaceRecognizer', 'Facemark', 'FacemarkAAM', 'FacemarkKazemi', 'FacemarkLBF', 'FacemarkTrain', 'FisherFaceRecognizer', 'FisherFaceRecognizer_create', 'LBPHFaceRecognizer', 'LBPHFaceRecognizer_create', 'MACE', 'MACE_create', 'MACE_load', 'PredictCollector', 'StandardCollector', 'StandardCollector_create', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_native', 'createFacemarkAAM', 'createFacemarkKazemi', 'createFacemarkLBF', 'drawFacemarks', 'getFacesHAAR', 'loadDatasetList', 'loadFacePoints', 'loadTrainingData']


In [3]:
import cv2

try:
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    print("LBPHFaceRecognizer is available.")
except AttributeError:
    print("LBPHFaceRecognizer is NOT available.")

LBPHFaceRecognizer is available.


In [45]:
import cv2
import os
import numpy as np

# Paths
dataset_path = "C:/Users/spurt/OneDrive/Desktop/dataset/dataset"  # Update with your dataset path
haar_cascade_path = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
model_path = "face_recognizer.yml"

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(haar_cascade_path)

# Function to prepare training data
def prepare_training_data(dataset_path):
    faces = []
    labels = []
    label_map = {}  # Map to store names with labels
    current_label = 0

    # Traverse the dataset directory
    for person_name in os.listdir(dataset_path):
        person_folder = os.path.join(dataset_path, person_name)

        if not os.path.isdir(person_folder):
            continue

        # Assign label to the person
        label_map[current_label] = person_name

        # Process each image in the person's folder
        for image_name in os.listdir(person_folder):
            image_path = os.path.join(person_folder, image_name)
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

            if img is None:
                print(f"Could not read image {image_path}. Skipping.")
                continue

            # Append face and label
            faces.append(img)
            labels.append(current_label)

        current_label += 1

    return faces, np.array(labels), label_map

# Train the model
def train_model():
    print("Preparing training data...")
    faces, labels, label_map = prepare_training_data(dataset_path)
    print(f"Training data prepared: {len(faces)} faces.")

    # Initialize LBPH recognizer
    recognizer = cv2.face.LBPHFaceRecognizer_create()

    # Train recognizer
    print("Training the recognizer...")
    recognizer.train(faces, labels)
    print("Training completed!")

    # Save the model
    recognizer.save(model_path)
    print(f"Model saved as {model_path}")
    return label_map

# Recognize the person in the test image
def recognize_face(test_image_path, label_map):
    # Load the trained recognizer
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read(model_path)

    # Read and process the test image
    img = cv2.imread(test_image_path)
    if img is None:
        raise FileNotFoundError(f"Could not load the test image: {test_image_path}")

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) == 0:
        print("No faces detected.")
        return

    for (x, y, w, h) in faces:
        face = gray[y:y + h, x:x + w]

        # Recognize face
        label, confidence = recognizer.predict(face)

        # Check if the label exists in the map
        if label in label_map:
            person_recognized = label_map[label]
        else:
            person_recognized = "Unknown"

        print(f"Recognized as: {person_recognized} (Confidence: {confidence})")

        # Draw rectangle and label
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(img, person_recognized, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Display the result
    cv2.imshow("Face Recognition", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Train the model and get the label map
label_map = train_model()

# Test the recognizer with a specific image
test_image_path = "C:/Users/spurt/OneDrive/Desktop/images (10).jpeg" # Update with your test image path
recognize_face(test_image_path, label_map)

Preparing training data...
Training data prepared: 75 faces.
Training the recognizer...
Training completed!
Model saved as face_recognizer.yml
Recognized as: Ratan Tata (Confidence: 95.276940369416)
